# LangGraph와 GraphRAG를 활용한 검색 및 활용 방안 및 사례
자연어 질의에 대해 중요 키워드를 추출하고, Dictionary 의미와 문서 내용을 비교해 검증하는 예제입니다.

이 과정이 가지는 중요한 의미는 유사도 또는 키워드 기반의 정보로 조회한 결과를 확장된 정보를 제공 할수 있다는 것 입니다.
특정 조회된 정보를 바탕으로 확장된 연결관계를 구조화 할때 얻어지는 정보는 단순히 키워드 기반의 정보 조회보다 훨씬 더 많은 정보를 제공 할 수 있습니다.
즉 해당인원의 프로젝트 수행이력, 연관된 사람들, 스팩, 투입된 프로젝트 수행 기간, 비용 등의 다양한 정보를 추가적으로 제안 및 제공 할 수 있습니다.

GraphDB를 활용한 Relation 중심의 정보를 초기 질의 정보에서 연관관계를 맺는 확장된 정보에 대한 제공
- 해당인원과 연관된 정보를 바탕으로 추가 질의 항목을 제안할 수 있는 방식으로 처리 가능함

In [ ]:
# 필수 패키지 설치
!pip install langgraph langchain openai neo4j pandas

In [ ]:
# LangGraph 예제 구성
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage
from langchain.chat_models import ChatOpenAI
from typing import TypedDict, List, Optional

llm = ChatOpenAI(model="gpt-4o")

class State(TypedDict):
    question: str
    keywords: List[str]
    definition: Optional[str]
    vector_docs: List[str]
    graph_docs: List[str]
    is_valid: bool
    final_answer: str

In [ ]:
# 노드 정의 함수들
def extract_keywords(state: State):
    prompt = f"다음 질문에서 핵심 키워드를 3개 이하로 추출해줘: '{state['question']}'"
    response = llm.invoke([HumanMessage(content=prompt)])
    keywords = response.content.replace("\n", "").split(", ")
    return {"keywords": keywords}

DICTIONARY = {
    "regularization": "과적합을 방지하기 위한 기법입니다.",
    "overfitting": "모델이 학습 데이터에 과하게 적응한 상태입니다."
}

def lookup_dictionary(state: State):
    for keyword in state["keywords"]:
        if keyword in DICTIONARY:
            return {"definition": DICTIONARY[keyword]}
    return {"definition": None}

def vector_search(state: State):
    return {"vector_docs": [f"'{kw}'에 대한 기술 문서 요약입니다." for kw in state["keywords"]]}

def validate_answer(state: State):
    prompt = (
        f"질문 키워드의 정의: {state['definition']}\n"
        f"검색된 문서 요약: {state['vector_docs'] + state['graph_docs']}\n"
        f"이 문서들이 키워드 정의와 일치합니까? 맞으면 true, 아니면 false"
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    is_valid = "true" in response.content.lower()
    return {"is_valid": is_valid}

def format_final_answer(state: State):
    if state["is_valid"]:
        return {"final_answer": f"키워드 '{state['keywords']}'에 대한 정확한 정보는 다음과 같습니다:\n\n{state['vector_docs'] + state['graph_docs']}"}
    else:
        return {"final_answer": "질문에 대한 정보는 사전에 정의된 의미와 일치하지 않습니다."}

In [ ]:
from neo4j import GraphDatabase

# Neo4j 드라이버 초기화
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "your_password"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def graph_rag(state: State):
    keywords = state["keywords"]
    docs = []

    with driver.session() as session:
        for keyword in keywords:
            # 예시 Cypher 쿼리: 'Keyword' 노드를 기준으로 연결된 문서 반환
            query = """
            MATCH (k:Keyword)-[:RELATED_TO]->(d:Document)
            WHERE toLower(k.name) = toLower($keyword)
            RETURN d.title AS title, d.summary AS summary
            LIMIT 3
            """
            results = session.run(query, keyword=keyword)
            for record in results:
                docs.append(f"[{record['title']}] {record['summary']}")

    return {"graph_docs": docs}

In [ ]:
# 그래프 정의 및 실행
graph = StateGraph(State)
graph.add_node("extract_keywords", extract_keywords)
graph.add_node("lookup_dictionary", lookup_dictionary)
graph.add_node("vector_search", vector_search)
graph.add_node("graph_rag", graph_rag)
graph.add_node("validate_answer", validate_answer)
graph.add_node("format_final_answer", format_final_answer)

graph.set_entry_point("extract_keywords")
graph.add_edge("extract_keywords", "lookup_dictionary")
graph.add_edge("lookup_dictionary", "vector_search")
graph.add_edge("vector_search", "graph_rag")
graph.add_edge("graph_rag", "validate_answer")
graph.add_edge("validate_answer", "format_final_answer")
graph.add_edge("format_final_answer", END)

runnable = graph.compile()

In [ ]:
# 실행 예시
result = runnable.invoke({"question": "regularization은 왜 필요한가요?"})
print(result["final_answer"])